In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
import numpy as np

print(f"tensorflow {tf.__version__}")
print(f"Numpy {np.__version__}")

In [114]:
def sampling(args):
	z_mean,z_log_var = args
	epsilon = K.random_normal(shape=(K.shape(z_mean)[0], LATENT_DIM),mean = 0.,stddev=epsilon_std)
	return z_mean + K.exp(z_log_var / 2) * epsilon

In [101]:
BATCH_SIZE = 100,
ORIGINAL_DIM = 28 * 28

LATENT_DIM = 2 # tHE cOMPRESSED dIMENSION
INTERMEDIATE_DIM = 256 # The units For Decoder
EPOCHS = 5
epsilon_std = 1.0 # For generating The Gaussian 


x = Input(shape=(ORIGINAL_DIM,), name="Input_Images")
h = Dense(INTERMEDIATE_DIM,activation=tf.nn.relu, name="Encoding")(x)
z_mean = Dense(LATENT_DIM, name="Mean")(h)
z_log_var = Dense(LATENT_DIM, name="Log_Variance")(h)
z = Lambda(sampling, output_shape=(LATENT_DIM,))([z_mean, z_log_var])
encoder = Model(x, [z_mean, z_log_var, z], name="Encoder")

In [102]:

input_decoder = Input(shape=(LATENT_DIM,), name='decoder_input')
decoder_h = Dense(INTERMEDIATE_DIM, activation=tf.nn.relu,name="decoder_h")(input_decoder)
input_decoded = Dense(ORIGINAL_DIM,activation=tf.nn.sigmoid,name="Flattened_decoded")(decoder_h)
decoder = Model(input_decoder,input_decoded,name="decoder")

#print(create_decoder().summary())
x,encoder = create_encoder()
print(x)
decoder = create_decoder()
output_combined = decoder(encoder(x)[2]) # Get The Z values, i.e The Latent Space reprsentation
vae = Model(x, output_combined)

vae = VAE()
#create_decoder().summary()

Tensor("Input_Images_48:0", shape=(None, 784), dtype=float32)
Tensor("Input_Images_49:0", shape=(None, 784), dtype=float32)
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Images (InputLayer)    [(None, 784)]             0         
_________________________________________________________________
Encoder (Model)              [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Model)              (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [118]:
def vae_loss(x : tf.Tensor, x_decoded_mean : tf.Tensor,
		z_log_var = z_log_var,z_mean = z_mean,
		original_dims = ORIGINAL_DIM):
	xent_loss = ORIGINAL_DIM * tf.keras.metrics.binary_crossentropy(x, x_decoded_mean)
	k1_loss = - 0.5 * tf.reduce_sum(
		1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=None)
	vae_lossd = tf.reduce_mean(xent_loss + k1_loss)
	return vae_lossd

vae.compile(optimizer="rmsprop",loss=vae_loss)
vae.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Images (InputLayer)    [(None, 784)]             0         
_________________________________________________________________
Encoder (Model)              [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Model)              (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [104]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [121]:
vae.fit((x_train, x_train),
        shuffle=True,
        epochs=EPOCHS,
        batch_size=64)

Train on 60000 samples
Epoch 1/5
   64/60000 [..............................] - ETA: 59s

IndexError: list index out of range